# Initialization

## Install Dependencies

In [1]:
# %%bash
# # keep only wikigraphs folder from deepmind-research repo
# if [ ! -d "wikigraphs" ]; then
#     git clone https://github.com/google-deepmind/deepmind-research.git
#     mv deepmind-research/wikigraphs/wikigraphs .
#     mv deepmind-research/wikigraphs/scripts wikigraphs  # move scripts folder inside wikigraphs module
# fi
# rm -rf deepmind-research
# exit

In [2]:
# %%bash
# PACKAGES="chromadb==0.6.3 jraph==0.0.6.dev0 tqdm==4.67.1 sentence-transformers==4.1.0 accelerate==1.7.0 huggingface_hub==0.33.0 spacy==3.8.6"
# if [[ -f "$(which mamba)" ]]; then
#     mamba install $PACKAGES -y
# elif [[ -f "$(which conda)" ]]; then
#     conda install -c conda-forge $PACKAGES -y
# else
#     pip install $PACKAGES
# fi

## Imports

In [3]:
import os
import json
import shutil
import requests
import tarfile
import zipfile
import gzip
import lzma

import chromadb
import ollama

import pandas as pd

from pathlib import Path
from itertools import chain
from tqdm.auto import tqdm
from IPython.display import display

from langchain.text_splitter import RecursiveCharacterTextSplitter

from wikigraphs.scripts.freebase_preprocess import pair_graphs_with_wikitext
from wikigraphs.data import io_tools, paired_dataset

## Configs

In [4]:
pd.set_option('display.max_colwidth', None)

## Global Variables

In [5]:
DATA_DIR = "data/"
FREEBASE_URLS = [
    ("https://docs.google.com/uc?export=download&id=1uuSS2o72dUCJrcLff6NBiLJuTgSU-uRo", "max256.tar"),
    ("https://docs.google.com/uc?export=download&id=1nOfUq3RUoPEWNZa2QHXl2q-1gA5F6kYh", "max512.tar"),
    ("https://docs.google.com/uc?export=download&id=1uuJwkocJXG1UcQ-RCH3JU96VsDvi7UD2", "max1024.tar")
]
FREEBASE_DIR = "data/freebase/"

WIKITEXT_URLS = [
    ("https://wikitext.smerity.com/wikitext-103-v1.zip", "wikitext-103-v1.zip"),
    ("https://wikitext.smerity.com/wikitext-103-raw-v1.zip", "wikitext-103-raw-v1.zip")
]

WIKITEXT_DIR = "data/wikitext-103"
WIKITEXT_RAW_DIR = WIKITEXT_DIR + "-raw"

WIKIGRAPHS_DATA_DIR = "data/wikigraphs/"

GENERATED_GRAPHS_SAMPLE_SIZE = 10

LLM_MODEL = "llama3.1:8b"

## Utilities

In [6]:
def chmod_recursive(path, mode):
    os.chmod(path, mode)
    for root, dirs, files in os.walk(path):
        for item in chain(dirs, files):
            os.chmod(os.path.join(root, item), mode)

def download_file(url, filepath):
    if os.path.exists(filepath):
        print(f"File {filepath} already exists. Skipping download.")
        return

    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(filepath, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

    print(f"File {filepath} downloaded successfully.")

def create_tar(source_item, save_path=".", compression=""):
    """Creates a tar archive of the specified item (file or folder)."""

    if compression not in ("gz", "bz2", "xz", ""):
        raise ValueError(f"Invalid compression method: {compression}")

    source_name = Path(source_item).name
    archive_name = f"{source_name}.tar{'.' + compression if compression else ''}"
    archive_path = f"{save_path}/{archive_name}"

    if os.path.exists(archive_path):
        print(f"Archive '{archive_name}' already exists. Skipping creation.")
        return

    try:
        mode = f"w:{compression}" if compression else "w"
        with tarfile.open(archive_path, mode) as tar:
            tar.add(source_item, arcname=source_name)
        print(f"Archive '{archive_name}' created successfully.")
    except FileNotFoundError as e:
        print(f"Error: Folder '{source_item}' not found.")
        raise e
    except Exception as e:
        print(f"An error occurred: {e}")
        raise e

def extract_tar(filename, save_path=".", under_new_folder=False):
    filepath = Path(save_path) / Path(filename)
    if not filepath.exists():
        raise ValueError(f"File `{filename}` does not exist under path `{save_path}`.")

    split = filename.rsplit(".", maxsplit=1)[-1]
    compression = split if split in ("gz", "bz2", "xz") else ""

    if split == "tar":
        compression = ""
    elif split in ("gz", "bz2", "xz"):
        compression = split
    else:
        raise ValueError(f"Invalid file extension: {split}")

    try:
        mode = f"r:{compression}" if compression else "r"
        with tarfile.open(filepath, mode) as tar:
            # properly remove .tar.XXX or .tar ext
            extract_folder_name = filename.rsplit(".", maxsplit=2 if compression else 1)[0]
            extract_folder = ( save_path if not under_new_folder else
                os.path.join(save_path, extract_folder_name) )
            os.makedirs(extract_folder, exist_ok=True)
            for member in tar:
                target_path = os.path.join(extract_folder, member.name)
                if os.path.exists(target_path):
                    # BUG: Displays Skipping even if they do not already exist
                    #      Must be regarding the way it loops through the members
                    # UPDATE: For some weird reason, this has stopped. Maybe
                    #         google colab local storage had an issue
                    print(f"Skipping {member.name}, already exists")
                else:
                    tar.extract(member, extract_folder)
            print(f"File {filename} extracted successfully.")
    except tarfile.ReadError as e:
        print(f"Error: Could not extract {filename}. It may not be a valid tar file.")
        raise e

def extract_zip(filename, save_path=".", under_new_folder=False):
    filepath = Path(save_path) / Path(filename)
    if not filepath.exists():
        raise ValueError(f"File `{filename}` does not exist under path `{save_path}`.")

    splits = filename.rsplit(".", maxsplit=1)
    if splits[-1] != "zip":
        raise ValueError(f"Invalid file extension: {splits[-1]}. Expected 'zip'")

    try:
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            extract_folder_name = splits[0]
            extract_folder = (save_path if not under_new_folder else
                              os.path.join(save_path, extract_folder_name))
            os.makedirs(extract_folder, exist_ok=True)

            for member_name in zip_ref.namelist():
                target_path = os.path.join(extract_folder, member_name)
                if os.path.exists(target_path):
                    print(f"Skipping {member_name}, already exists")
                else:
                    zip_ref.extract(member_name, extract_folder)
            print(f"File {filename} extracted successfully.")
    except zipfile.BadZipFile as e:
        print(f"Error: Could not extract {filename}. It may not be a valid zip file.")
        raise e

# Loading

In [7]:
def concatenate(input_files, output_file, compress=True):
    if os.path.exists(output_file):
        print(f"File {output_file} already exists. Skipping concatenation.")
    else:
        with open(output_file, 'w') as outfile:
            for input_file in input_files:
                with open(input_file, 'r') as infile:
                    outfile.write(infile.read())
        print(f"Concatenated {input_files} files into {output_file}")

    if compress:
        with open(output_file, 'rb') as f_in:
            compressed_file = output_file + '.gz'
            if os.path.exists(compressed_file):
                print(f"File {compressed_file} already exists. Skipping compression.")
                return
            with gzip.open(compressed_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
                # f_out.writelines(f_in)  # alternative
                print(f"Compressed {output_file} into {compressed_file}")

def extract_dataset(filename, save_path=".", merge_dataset_types=True):
    extract_tar(filename, save_path=save_path, under_new_folder=True)
    extract_folder = filename.rsplit(save_path, maxsplit=1)[0]

    # Extract gz files within the extracted folder
    for root, _, files in os.walk(extract_folder):
        extracted_file_paths = []
        for file in files:
            if file.endswith(".gz"):
                gz_file_path = os.path.join(root, file)
                try:
                    with gzip.open(gz_file_path, 'rb') as gz_file:
                        extracted_file_path = gz_file_path[:-3] # remove .gz extension
                        if os.path.exists(extracted_file_path):
                            print(f"File {extracted_file_path} already exists. Skipping extraction.")
                            continue
                        with open(extracted_file_path, 'wb') as extracted_file:
                            extracted_file.write(gz_file.read())
                    extracted_file_paths.append(extracted_file_path)
                    print(f"Extracted: {gz_file_path} to {extracted_file_path}")
                except gzip.BadGzipFile as e:
                    print(f"Error: Could not extract {gz_file_path}. It may not be a valid gzip file.")
                    raise e
        if merge_dataset_types:
            concatenate(extracted_file_paths, os.path.join(extract_folder, "whole"))
        break  # in case subfolders exist inside max1024 it won't go deeper to search them (i.e forces only one iteration)

## Freebase Data

In [8]:
os.makedirs(FREEBASE_DIR, exist_ok=True)

for url, filename in FREEBASE_URLS:
    download_file(url, FREEBASE_DIR + filename)
    extract_dataset(filename, save_path=FREEBASE_DIR)

File data/freebase/max256.tar already exists. Skipping download.
Skipping test.gz, already exists
Skipping train.gz, already exists
Skipping valid.gz, already exists
File max256.tar extracted successfully.
File data/freebase/max512.tar already exists. Skipping download.
Skipping test.gz, already exists
Skipping train.gz, already exists
Skipping valid.gz, already exists
File max512.tar extracted successfully.
File data/freebase/max1024.tar already exists. Skipping download.
Skipping test.gz, already exists
Skipping train.gz, already exists
Skipping valid.gz, already exists
File max1024.tar extracted successfully.


## WikiText-103 Data

In [9]:
for url, filename in WIKITEXT_URLS:
    download_file(url, DATA_DIR + filename)
    extract_zip(filename, save_path=DATA_DIR)

File data/wikitext-103-v1.zip already exists. Skipping download.
Skipping wikitext-103/, already exists
Skipping wikitext-103/wiki.test.tokens, already exists
Skipping wikitext-103/wiki.valid.tokens, already exists
Skipping wikitext-103/wiki.train.tokens, already exists
File wikitext-103-v1.zip extracted successfully.
File data/wikitext-103-raw-v1.zip already exists. Skipping download.
Skipping wikitext-103-raw/, already exists
Skipping wikitext-103-raw/wiki.test.raw, already exists
Skipping wikitext-103-raw/wiki.valid.raw, already exists
Skipping wikitext-103-raw/wiki.train.raw, already exists
File wikitext-103-raw-v1.zip extracted successfully.


## Creation of WikiGraphs Dataset
which is a Text-to-Graph dataset, pairing the Freebase subgraphs with their corresponding WikiText-103

In [10]:
for subset in ['train', 'valid', 'test']:
    pair_graphs_with_wikitext(subset,
                              WIKITEXT_DIR,
                              FREEBASE_DIR + "max256/",
                              WIKIGRAPHS_DATA_DIR + "max256/")

## WikiGraphs Dataset (a.k.a Paired Dataset)

In [11]:
paired_dataset.DATA_ROOT = WIKIGRAPHS_DATA_DIR
dataset = paired_dataset.ParsedDataset(
               subset = 'train',
               shuffle_data = False,
               data_dir = None,
               version = 'max256')

In [12]:
parsed_pairs = list(dataset)

In [13]:
len(parsed_pairs)

23431

In [14]:
pair = parsed_pairs[0]

### Parsed Pair Example

In [15]:
example_pair = parsed_pairs[0]

In [16]:
example_pair.center_node

'ns/m.0ddd390'

In [17]:
example_pair.title

'Valkyria_Chronicles_III'

In [18]:
print(example_pair.text[:1000])


 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series n

In [19]:
example_pair.graph.edges()[:10]

[(0, 1, 'key/wikipedia.en'),
 (0, 2, 'ns/type.object.name'),
 (0, 3, 'ns/organization.organization.date_founded'),
 (0, 4, 'ns/cvg.cvg_developer.games_developed'),
 (0, 5, 'ns/common.topic.description'),
 (6, 7, 'ns/type.object.name'),
 (6, 8, 'key/wikipedia.en'),
 (6, 4, 'ns/cvg.cvg_genre.games'),
 (6, 9, 'ns/common.topic.description'),
 (10, 11, 'key/wikipedia.en')]

__Note__: One needs to use `graph.nodes()` to get the node freebase id from the node index id by indexing on the returned list

# Preprocessing

## Ground Truth Graphs

In [20]:
truth_graphs_dfs = []  # ground truth graph dataframes
truth_graphs_dfs_dict = {}  # center_node - ground truth graph dataframe pairs

for pair in parsed_pairs:
    g = pair.graph
    df = pd.DataFrame(g.edges(), columns=["src", "tgt", "edge"])
    # NOTE: "src" corresponds to the ID of the "subject" node and
    #       "tgt" corresponds to the ID of the "object" node.
    
    # Append the values of the nodes as new columns
    df["subject"] = df["src"].apply(lambda node_id: g.nodes()[node_id])
    df["object"] = df["tgt"].apply(lambda node_id: g.nodes()[node_id])

    # Creating Subject-Predicate-Object only columns dataframe
    df = df[["subject", "edge", "object"]]
    df.rename(columns={"edge":"predicate"}, inplace=True)

    truth_graphs_dfs.append(df)
    truth_graphs_dfs_dict[pair.center_node] = df

# Concatenate the dfs using the center node of each df as index value for all its rows
truth_df = pd.concat(truth_graphs_dfs_dict, names=["center_node"])

### Examples

In [21]:
truth_df.head()

subject                                  predicate  \
center_node                                                              
ns/m.0ddd390 0  ns/m.0f9q9z                           key/wikipedia.en   
             1  ns/m.0f9q9z                        ns/type.object.name   
             2  ns/m.0f9q9z  ns/organization.organization.date_founded   
             3  ns/m.0f9q9z       ns/cvg.cvg_developer.games_developed   
             4  ns/m.0f9q9z                ns/common.topic.description   

                                                                          object  
center_node                                                                       
ns/m.0ddd390 0                                                        "Sega_AM1"  
             1                                                        "Sega Wow"  
             2                                                            "2000"  
             3                                                      ns/m.0ddd390  
             4  "Sega Wow was a division of Japanese video game developer Sega."

In [22]:
truth_df.loc["ns/m.0ddd390"].head()

,subject,predicate,object
0,ns/m.0f9q9z,key/wikipedia.en,"""Sega_AM1"""
1,ns/m.0f9q9z,ns/type.object.name,"""Sega Wow"""
2,ns/m.0f9q9z,ns/organization.organization.date_founded,"""2000"""
3,ns/m.0f9q9z,ns/cvg.cvg_developer.games_developed,ns/m.0ddd390
4,ns/m.0f9q9z,ns/common.topic.description,"""Sega Wow was a division of Japanese video game developer Sega."""


### Freebase Sub-Graphs Dataframe Creation

In [23]:
def create_dataframe(graphs):
    df_list = []

    for g in graphs:

        df = pd.DataFrame(g.edges(), columns=["src", "tgt", "edge"])
        # NOTE: "src" corresponds to the ID of the "subject" node and
        #       "tgt" corresponds to the ID of the "object" node.
        # Append the values of the nodes as new columns

        df["subject"] = df["src"].apply(lambda node_id: g.nodes()[node_id])
        df["object"] = df["tgt"].apply(lambda node_id: g.nodes()[node_id])

        spo_df = df[["subject", "edge", "object"]].copy()
        spo_df.rename(columns={"edge":"predicate"}, inplace=True)

        df_list.append(spo_df)

    return pd.concat(df_list, ignore_index=True)

In [24]:
freebase_graphs_generator = io_tools.graphs_from_file("data/freebase/max1024/whole.gz")
freebase_graphs = list(freebase_graphs_generator)
len(freebase_graphs)

23851

In [25]:
# Convert from io_tools.Graph to paired_dataset.Graph
freebase_graphs = [paired_dataset.Graph.from_edges(g.edges) for g in freebase_graphs]

In [26]:
freebase_df = create_dataframe(freebase_graphs)

#### Show Duplicates

In [27]:
freebase_df_dups = freebase_df[freebase_df.duplicated(keep=False)].sort_values(["subject", "predicate", "object"])
display(freebase_df_dups)

,subject,predicate,object
351415,ns/g.113qbncfz,ns/common.topic.description,"""Rock & Alternative French Radio | www.lagrosseradio.com"""
834396,ns/g.113qbncfz,ns/common.topic.description,"""Rock & Alternative French Radio | www.lagrosseradio.com"""
351416,ns/g.113qbncfz,ns/type.object.name,"""LA GROSSE RADIO"""
834398,ns/g.113qbncfz,ns/type.object.name,"""LA GROSSE RADIO"""
494921,ns/g.11b6mp5yt4,ns/common.topic.description,"""Katie Bell is a fictional character from the Harry Potter book series written by J. K. Rowling."""
...,...,...,...
155920,ns/m.0zz4ncc,ns/type.object.name,"""Battle"""
156465,ns/m.0zz4ncc,ns/type.object.name,"""Battle"""
156891,ns/m.0zz4ncc,ns/type.object.name,"""Battle"""
157409,ns/m.0zz4ncc,ns/type.object.name,"""Battle"""


#### Remove Duplicates

For some reason, the max1024 version of freebase has duplicates inside. These are removed below.

TODO: Find the reason why duplicates exist

In [28]:
# Before duplicates removal
freebase_df_len_before_dups_removal = len(freebase_df)
print(freebase_df_len_before_dups_removal)

1312745


In [29]:
freebase_df.drop_duplicates(inplace=True)

In [30]:
# After duplicates removal
freebase_df_len_after_dups_removal = len(freebase_df)
print(freebase_df_len_after_dups_removal)

743965


In [31]:
# Duplicates removed
freebase_df_dups_removed = freebase_df_len_before_dups_removal - freebase_df_len_after_dups_removal
print(freebase_df_dups_removed)

568780


## Unique Entity Names

In [32]:
object_name_df = freebase_df[freebase_df["predicate"] == "ns/type.object.name"].drop_duplicates()

# List of unique enitity names, defined as the value (object) of the
# "ns/type.object.name" (predicate) of the corresponding entity's ID (subject)
unique_enitity_names = object_name_df["object"].drop_duplicates().tolist()

In [33]:
len(unique_enitity_names)

81296

### Unique Entity ID to Name and vice-versa Dictionaries

In [34]:
id_to_name = object_name_df.set_index("subject")["object"].to_dict()

In [35]:
print("Number of unique IDs:")
print(len(id_to_name))

Number of unique IDs:
92128


In [36]:
name_to_id = {}
for _id, name in id_to_name.items():
    name_to_id.setdefault(name, []).append(_id)

In [37]:
print("Number of unique names:")
print(len(name_to_id))

Number of unique names:
81296


In [38]:
name_to_id_1to1 = {k: v[0] for k, v in name_to_id.items() if len(v) == 1}

In [39]:
print("Number of unique names that correspond to single entity ID:")
print(len(name_to_id_1to1))

Number of unique names that correspond to single entity ID:
76885


In [40]:
name_to_id_1toN = {k: v for k, v in name_to_id.items() if len(v) > 1}

In [41]:
print("Number of unique names that correspond to multiple entity IDs:")
print(len(name_to_id_1toN))

Number of unique names that correspond to multiple entity IDs:
4411


In [42]:
name_to_id_1toN

{'"Millennium"': ['ns/m.036qs6', 'ns/m.03ydg8b'],
 '"Closure"': ['ns/m.02nw7wx', 'ns/m.04f_98r', 'ns/m.047jr2'],
 '"Oregon"': ['ns/m.05kj_', 'ns/m.0k2lfmq', 'ns/m.0sf4d'],
 '"Portland"': ['ns/m.02frhbc', 'ns/m.0c4kv', 'ns/m.0smnk'],
 '"Manchester"': ['ns/m.052bw', 'ns/m.0xhj2'],
 '"Charleston"': ['ns/m.0gkgp', 'ns/m.0fw2f'],
 '"Japan"': ['ns/m.03_3d', 'ns/m.01c7s7f'],
 '"Athlete"': ['ns/m.02h7c7b', 'ns/m.01445t', 'ns/m.02gr2z'],
 '"Profession"': ['ns/m.02h67g7', 'ns/m.02h65n3'],
 '"Celebrity"': ['ns/m.02_7frv', 'ns/m.06z14x_'],
 '"Height"': ['ns/m.025d7wc', 'ns/m.0wjcd9q'],
 '"Bourbon County"': ['ns/m.0nq49', 'ns/m.0nnv5'],
 '"Species"': ['ns/m.06zf0', 'ns/m.032_76'],
 '"Initial release date"': ['ns/m.0jsg52', 'ns/m.0kpv77', 'ns/m.021y6pg'],
 '"Comedy"': ['ns/m.05p553', 'ns/m.01m7g01'],
 '"Politician"': ['ns/m.0fj9f', 'ns/m.02xlh55'],
 '"Lake County"': ['ns/m.0p028', 'ns/m.0nvd8'],
 '"Papa Stour"': ['ns/m.02ftdw', 'ns/m.029wlqr'],
 '"SpongeBob SquarePants"': ['ns/m.07vqnc', 'ns/m.01b9t

#### Examples of Unique Names to Multiple Entity IDs

**Athlete Example**

In [43]:
example_name_1toN = "Athlete"
example_ids_1toN = name_to_id_1toN[f'"{example_name_1toN}"']
print(example_ids_1toN)

['ns/m.02h7c7b', 'ns/m.01445t', 'ns/m.02gr2z']


In [44]:
# Data Exploration
for example_id in example_ids_1toN:
    display(freebase_df.query(f'subject=="{example_id}"'))

,subject,predicate,object
475,ns/m.02h7c7b,ns/type.object.name,"""Athlete"""
476,ns/m.02h7c7b,ns/common.topic.description,"""An athlete is a person who participates in a sport for personal fitness or as part of an amateur or professional competition or contest. \nFor more information, please see the Freebase wiki page on Athlete."""
477,ns/m.02h7c7b,ns/freebase.type_profile.instance_count,"""271978"""
478,ns/m.02h7c7b,ns/type.object.id,"""/sports/pro_athlete"""
479,ns/m.02h7c7b,ns/freebase.object_hints.best_hrid,"""/sports/pro_athlete"""


,subject,predicate,object
33537,ns/m.01445t,ns/common.topic.alias,"""Sportsman"""
33538,ns/m.01445t,ns/common.topic.description,"""An athlete or sportsperson, sportsman or sportswoman is a person who competes in one or more sports that involve physical strength, speed and/or endurance. Athletes may be professionals or amateurs.\nMost professional athletes have particularly well-developed physiques obtained by extensive physical training and strict exercise accompanied by a strict dietary regimen.\nThe word \""athlete\"" is a romanization of the Greek: άθλητὴς, athlētēs, one who participates in a contest; from ἄθλος, áthlos, or ἄθλον, áthlon, a contest or feat. It is the general term for all participants in any physical sport; its application to those who participate in other activities, such as horse riding or driving, is somewhat controversial."""
33539,ns/m.01445t,key/wikipedia.en,"""Athlete"""
33540,ns/m.01445t,ns/type.object.name,"""Athlete"""
33541,ns/m.01445t,ns/people.profession.people_with_this_profession,ns/m.02r6pqd
...,...,...,...
1298324,ns/m.01445t,ns/people.profession.people_with_this_profession,ns/m.01jzhl
1305767,ns/m.01445t,ns/people.profession.people_with_this_profession,ns/m.03gzkgc
1305924,ns/m.01445t,ns/people.profession.people_with_this_profession,ns/m.08887m
1309808,ns/m.01445t,ns/people.profession.people_with_this_profession,ns/m.0j43qj4


,subject,predicate,object
316187,ns/m.02gr2z,key/wikipedia.en,"""Athlete_$0028group$0029"""
316188,ns/m.02gr2z,ns/music.artist.origin,ns/m.02z6bk
316189,ns/m.02gr2z,ns/common.topic.alias,"""Athelete"""
316190,ns/m.02gr2z,ns/music.artist.active_start,"""2002-03"""
316191,ns/m.02gr2z,ns/common.topic.description,"""Athlete are an English indie rock band formed in Deptford, London, comprising Joel Pott, Carey Willetts, Stephen Roberts and Tim Wanstall.\nThe band had a brief period of high-profile domestic success in which their debut album Vehicles & Animals was a platinum seller in 2005 and Mercury Music Prize nomination. It was followed up by Tourist which reached No. 1 and sold over 500,000 copies allowing this album to also go platinum. Since then the band has continued to release records on regular basis. Their subsequent two albums Beyond the Neighbourhood and Black Swan sees the band exploring different styles, influenced by the works of artists such as Nick Cave & the Bad Seeds."""
316192,ns/m.02gr2z,ns/type.object.name,"""Athlete"""


**Observation**: As one can see from the above example for the name "Athlete", the first and second entities are of the same meaning while the third one has a different meaning (i.e rock band instead of the sportsperson)

**SpongeBob SquarePants Example**

In [45]:
example_name_1toN = "SpongeBob SquarePants"
example_ids_1toN = name_to_id_1toN[f'"{example_name_1toN}"']
print(example_ids_1toN)

['ns/m.07vqnc', 'ns/m.01b9tt']


In [46]:
# Data Exploration
for example_id in example_ids_1toN:
    display(freebase_df.query(f'subject=="{example_id}"'))

,subject,predicate,object
1373,ns/m.07vqnc,key/wikipedia.en,"""SpongeBob"""
1374,ns/m.07vqnc,ns/tv.tv_program.number_of_episodes,"""355"""
1375,ns/m.07vqnc,ns/common.topic.description,"""SpongeBob SquarePants is an American animated television series created by marine biologist and animator Stephen Hillenburg for Nickelodeon. The series chronicles the adventures and endeavors of the title character and his various friends in the fictional underwater city of Bikini Bottom. The series' popularity has made it a media franchise, as well as the highest rated series to ever air on Nickelodeon, and the most distributed property of MTV Networks. The media franchise has generated $8 billion in merchandising revenue for Nickelodeon.\nMany of the ideas for the series originated in an unpublished, educational comic book titled The Intertidal Zone, which Hillenburg created in the mid-1980s. He began developing SpongeBob SquarePants into a television series in 1996 upon the cancellation of Rocko's Modern Life, and turned to Tom Kenny, who had worked with him on that series, to voice the titular character. SpongeBob was originally going to be named SpongeBoy, and the series was to be called SpongeBoy Ahoy!, but these were both changed, as the name was already trademarked."""
1376,ns/m.07vqnc,ns/tv.tv_program.air_date_of_first_episode,"""1999-05-01"""
1377,ns/m.07vqnc,ns/tv.tv_program.seasons,ns/m.043rdbm
1378,ns/m.07vqnc,ns/tv.tv_program.episodes,ns/m.07749b
1379,ns/m.07vqnc,ns/common.topic.alias,"""SpongeBob"""
1380,ns/m.07vqnc,ns/tv.tv_program.episode_running_time,"""30"""
1381,ns/m.07vqnc,ns/tv.tv_program.currently_in_production,"""true"""
1382,ns/m.07vqnc,ns/tv.tv_program.number_of_seasons,"""9"""


,subject,predicate,object
1044310,ns/m.01b9tt,key/wikipedia.en,"""SpongeBob_SquarePants_$0028character$0029"""
1044311,ns/m.01b9tt,ns/common.topic.notable_types,ns/m.02nsjl9
1044312,ns/m.01b9tt,ns/common.topic.description,"""SpongeBob SquarePants is the titular character and protagonist of the American animated television series of the same name. He is voiced by actor and comedian Tom Kenny, and first appeared on television in the series' pilot episode \""Help Wanted\"" on May 1, 1999.\nSpongeBob SquarePants was created and designed by cartoonist and marine biologist Stephen Hillenburg shortly after the cancellation of Rocko's Modern Life in 1996. Hillenburg intended to create a series about an over-optimistic sponge that annoys other characters. Hillenburg compared the concept to Laurel and Hardy and Pee-wee Herman. As he drew the character, he decided that a \""squeaky-clean square\"" fit the concept. His name is derived from \""Bob the Sponge\"", the host of Hillenburg's comic strip The Intertidal Zone that he originally drew in the 1980s while teaching marine biology to visitors of the Ocean Institute. SpongeBob is a naïve and goofy sea sponge who works as a fry cook in the fictional underwater town of Bikini Bottom.\nThe character has received positive critical response from media critics and achieved popularity with both children and adults, though he has been involved in public controversy."""
1044313,ns/m.01b9tt,ns/fictional_universe.fictional_character.gender,ns/m.05zppz
1044314,ns/m.01b9tt,ns/type.object.name,"""SpongeBob SquarePants"""


**Observation**: As one can see from the above example for the name "SpongeBob SquarePants", the first and second entities are different. One being the show and the other being the name of the character which the show got its name from. In this example, we do not have different entities with the same meaning.

**Pilot Example**

In [47]:
example_name_1toN = "Pilot"
example_ids_1toN = name_to_id_1toN[f'"{example_name_1toN}"']
print(example_ids_1toN)

['ns/m.04m_mcw', 'ns/m.05dpjrg', 'ns/m.0h7mfg6', 'ns/m.0kffn39', 'ns/m.0bwlvv', 'ns/m.0kbdl4b', 'ns/m.0j3fv7g', 'ns/m.06yznhd', 'ns/m.0dkt96', 'ns/m.05syyz4', 'ns/m.0kxflf7', 'ns/m.07mmh1y', 'ns/m.03d6l9d', 'ns/m.0gzb7m', 'ns/m.0fqp_h', 'ns/m.0c0293', 'ns/m.02qnz0j', 'ns/m.011nsb7_', 'ns/m.0c3vwgr', 'ns/m.02nw2nc', 'ns/m.0dpg5_', 'ns/m.0krtwc0', 'ns/m.0795xq1', 'ns/m.0gj8cyj', 'ns/m.05dqwkp', 'ns/m.05dpzp8', 'ns/m.05dtp0f', 'ns/m.0mwpj6b', 'ns/m.0260p1z', 'ns/m.0x0n5mq', 'ns/m.0808gzp', 'ns/m.090ldx', 'ns/m.0w5zm5g', 'ns/m.07kf5_9', 'ns/m.0ddh9q1', 'ns/m.0269nfn', 'ns/m.0bzd9w', 'ns/m.07k8c34', 'ns/m.0hgpv3v', 'ns/m.05q88m9', 'ns/m.0gjdv5', 'ns/m.0kb0_g3', 'ns/m.01xwlgy', 'ns/m.025z3j0', 'ns/m.0bjtfn', 'ns/m.0hnb4n5', 'ns/m.06gt51c', 'ns/m.0fcygc', 'ns/m.05dlk4d', 'ns/m.06yp_4v', 'ns/m.0jfyh85', 'ns/m.0703lxy', 'ns/m.0kd3pbb']


In [48]:
# Data Exploration
for example_id in example_ids_1toN:
    display(freebase_df.query(f'subject=="{example_id}"'))

,subject,predicate,object
1708,ns/m.04m_mcw,ns/tv.tv_series_episode.next_episode,ns/m.0g9x0r3
1709,ns/m.04m_mcw,ns/tv.tv_series_episode.writer,ns/m.09pl3f
1710,ns/m.04m_mcw,ns/common.topic.description,"""When all the passengers on a plane die, FBI agent Olivia Dunham investigates the events and her partner almost dies. A desperate Olivia looks for help from Dr. Walter Bishop who has been institutionalized. Olivia, Dr. Bishop and his son Peter begin to discover what really happened on Flight 627 and begin to uncover a larger truth."""
1711,ns/m.04m_mcw,ns/tv.tv_series_episode.air_date,"""2008-09-09"""
1712,ns/m.04m_mcw,ns/type.object.name,"""Pilot"""
1713,ns/m.04m_mcw,ns/tv.tv_series_episode.season,ns/m.06yzmjr
1714,ns/m.04m_mcw,ns/common.topic.notable_types,ns/m.01xrzlb
1715,ns/m.04m_mcw,ns/tv.tv_series_episode.series,ns/m.03cf9ly
1716,ns/m.04m_mcw,key/wikipedia.en,"""Pilot_$0028Fringe$0029"""
1717,ns/m.04m_mcw,ns/tv.tv_series_episode.episode_number,"""1"""


,subject,predicate,object
51009,ns/m.05dpjrg,ns/tv.tv_series_episode.season_number,"""1"""
51010,ns/m.05dpjrg,ns/tv.tv_series_episode.series,ns/m.02tp4s
51011,ns/m.05dpjrg,ns/tv.tv_series_episode.production_number,"""101"""
51012,ns/m.05dpjrg,ns/common.topic.description,"""Film critic Jay Sherman falls in love with actress Valerie Fox. Jay's family and friends worry that Valerie is using him to get a good movie review. Does she truly care for him?"""
51013,ns/m.05dpjrg,ns/tv.tv_series_episode.air_date,"""1994-01-26"""
51014,ns/m.05dpjrg,ns/tv.tv_series_episode.writer,ns/m.03xpf_7
51015,ns/m.05dpjrg,key/wikipedia.en,"""Pilot_$0028The_Critic$0029"""
51016,ns/m.05dpjrg,ns/tv.tv_series_episode.episode_number,"""1"""
51017,ns/m.05dpjrg,ns/type.object.name,"""Pilot"""


,subject,predicate,object
71427,ns/m.0h7mfg6,ns/tv.tv_series_episode.director,ns/m.07g7h2
71428,ns/m.0h7mfg6,ns/tv.tv_series_episode.episode_number,"""1"""
71429,ns/m.0h7mfg6,ns/common.topic.description,"""A therapist and his family move across the country to escape their troubled past but quickly discover that their new home comes with its own horrific baggage."""
71430,ns/m.0h7mfg6,ns/type.object.name,"""Pilot"""
71431,ns/m.0h7mfg6,ns/tv.tv_series_episode.next_episode,ns/m.0h989bt
71432,ns/m.0h7mfg6,ns/tv.tv_series_episode.writer,ns/m.07g7h2
71433,ns/m.0h7mfg6,ns/tv.tv_series_episode.season_number,"""1"""
71434,ns/m.0h7mfg6,ns/tv.tv_series_episode.season,ns/m.0kcvvyh
71435,ns/m.0h7mfg6,ns/tv.tv_series_episode.series,ns/m.0h3mh3q
71436,ns/m.0h7mfg6,ns/tv.tv_series_episode.air_date,"""2011-10-05"""


,subject,predicate,object
92533,ns/m.0kffn39,ns/tv.tv_series_episode.episode_number,"""1"""
92534,ns/m.0kffn39,ns/tv.tv_series_episode.season_number,"""1"""
92535,ns/m.0kffn39,ns/tv.tv_series_episode.series,ns/m.0c2c5c
92536,ns/m.0kffn39,ns/common.topic.description,"""Arthur \""AC\"" Curry is a keen environmentalist with secret abilities that are heightened when he is around water. Upon learning of his true identity Arthur begins to investigate why random people who disappeared in the Bermuda Triangle years earlier are suddenly beginning to resurface."""
92537,ns/m.0kffn39,ns/type.object.name,"""Pilot"""
92538,ns/m.0kffn39,ns/tv.tv_series_episode.air_date,"""2006-07-24"""


,subject,predicate,object
97088,ns/m.0bwlvv,ns/common.topic.description,"""Meet the Bluths. In the pilot, Michael decides to take a job in Arizona after being passed over by George Sr. for head of the Bluth Company. George Sr. is jailed, and the family realizes they need Michael."""
97089,ns/m.0bwlvv,ns/tv.tv_series_episode.season_number,"""1"""
97090,ns/m.0bwlvv,ns/type.object.name,"""Pilot"""
97091,ns/m.0bwlvv,ns/tv.tv_series_episode.writer,ns/m.06brp0
97092,ns/m.0bwlvv,ns/tv.tv_series_episode.next_episode,ns/m.0bpvy2
97093,ns/m.0bwlvv,key/wikipedia.en,"""Extended_Pilot"""
97094,ns/m.0bwlvv,ns/tv.tv_series_episode.air_date,"""2003-11-02"""
97095,ns/m.0bwlvv,ns/tv.tv_series_episode.series,ns/m.02hct1
97096,ns/m.0bwlvv,ns/tv.tv_series_episode.episode_number,"""1"""
97097,ns/m.0bwlvv,ns/tv.tv_series_episode.season,ns/m.02kzkl_


...

,subject,predicate,object
1061250,ns/m.05dlk4d,ns/tv.tv_series_episode.season_number,"""1"""
1061251,ns/m.05dlk4d,ns/type.object.name,"""Pilot"""
1061252,ns/m.05dlk4d,ns/common.topic.description,"""Sam, Billie, Adam and Dan are four New Yorkers who are dealing with their individual eating disorders together. All four of them lean on one another for support and are good friends. They attend Belttighteners, which is a recovery program that approaches their issues extremely ruthless ways. Sam is trying to shape his date into a woman from a hot TV commercial, which horrifies Billie. Dan struggles with his choice for a gastric bypass surgery and Adam chases down a deliveryman for food."""
1061253,ns/m.05dlk4d,ns/tv.tv_series_episode.director,ns/m.0fh314
1061254,ns/m.05dlk4d,ns/tv.tv_series_episode.air_date,"""2005-08-04"""
1061255,ns/m.05dlk4d,ns/tv.tv_series_episode.writer,ns/m.0fh314
1061256,ns/m.05dlk4d,ns/tv.tv_series_episode.production_number,"""101"""
1061257,ns/m.05dlk4d,ns/tv.tv_series_episode.series,ns/m.07czvc
1061258,ns/m.05dlk4d,ns/tv.tv_series_episode.next_episode,ns/m.05dlk49
1061259,ns/m.05dlk4d,ns/tv.tv_series_episode.episode_number,"""1"""


,subject,predicate,object
1101622,ns/m.06yp_4v,ns/tv.tv_series_episode.writer,ns/m.09hd16
1101623,ns/m.06yp_4v,ns/tv.tv_series_episode.next_episode,ns/m.06yp_54
1101624,ns/m.06yp_4v,ns/tv.tv_series_episode.series,ns/m.01ch04
1101625,ns/m.06yp_4v,ns/tv.tv_series_episode.episode_number,"""1"""
1101626,ns/m.06yp_4v,ns/tv.tv_series_episode.season_number,"""1"""
1101627,ns/m.06yp_4v,ns/tv.tv_series_episode.air_date,"""1993-08-27"""
1101628,ns/m.06yp_4v,ns/common.topic.description,"""When U.S. Marshal Brisco County, Sr. is murdered by John Bly and his gang of outlaws, the robber barons of San Francisco's Westerfield Club hire Brisco County, Jr. as a bounty hunter to round them up. Brisco meets a lot of interesting people along the way, including rival bounty hunter Lord Bowler, and the Westerfield Club's stuffy lawyer, Socrates Poole. He also learns of a mysterious object which John Bly would do anything to possess, for the supernatural power it can give him."""
1101629,ns/m.06yp_4v,ns/type.object.name,"""Pilot"""


,subject,predicate,object
1107779,ns/m.0jfyh85,ns/common.topic.description,"""With his father away at the Crusades, Edmund comes up with a plan to prove his brother is illegitimate, thus making him Prince Regent.\n\nThe Blackadder pilot was shot but never aired on terrestrial TV in the UK (although some scenes were shown in the 25th anniversary special Blackadder Rides Again). One notable difference in the pilot, as in many pilots, is the casting. Baldrick is played not by Tony Robinson, but by Philip Fox. Another significant difference is that the character of Prince Edmund presented in the pilot is much closer to the intelligent, conniving Blackadder of the later series than the sniveling, weak Edmund of the original series. Set in the year 1582, the script of the pilot is roughly the same as the episode \""Born to be King\"", albeit with some different jokes, with some lines appearing in other episodes of the series."""
1107780,ns/m.0jfyh85,ns/tv.tv_series_episode.series,ns/m.01d25
1107781,ns/m.0jfyh85,ns/common.topic.alias,"""Original Pilot"""
1107782,ns/m.0jfyh85,ns/type.object.name,"""Pilot"""
1107783,ns/m.0jfyh85,ns/tv.tv_series_episode.season_number,"""0"""
1107784,ns/m.0jfyh85,ns/tv.tv_series_episode.episode_number,"""1"""


,subject,predicate,object
1119544,ns/m.0703lxy,ns/tv.tv_series_episode.series,ns/m.0642f_4
1119545,ns/m.0703lxy,ns/common.topic.description,"""Dylan is given an impossible pro bono custody case, Beth keeps mum when a 90-year-old man signs an agreement thinking that she's his daughter, Liam must get a client to sign with the firm under false pretenses, Addy finally gets some attention from her boss when she speaks her mind, and Malcolm gets off on the wrong foot when he's hired outside of the firm's traditional process."""
1119546,ns/m.0703lxy,ns/tv.tv_series_episode.air_date,"""2010-01-21"""
1119547,ns/m.0703lxy,ns/type.object.name,"""Pilot"""
1119548,ns/m.0703lxy,ns/tv.tv_series_episode.season_number,"""1"""
1119549,ns/m.0703lxy,ns/tv.tv_series_episode.next_episode,ns/m.09rmkb1
1119550,ns/m.0703lxy,ns/tv.tv_series_episode.episode_number,"""1"""


,subject,predicate,object
1119630,ns/m.0kd3pbb,ns/type.object.name,"""Pilot"""
1119631,ns/m.0kd3pbb,ns/tv.tv_series_episode.episode_number,"""1"""
1119632,ns/m.0kd3pbb,ns/tv.tv_series_episode.series,ns/m.0642f_4
1119633,ns/m.0kd3pbb,ns/common.topic.description,"""Dylan is given an impossible pro bono custody case, Beth keeps mum when a 90-year-old man signs an agreement thinking that she's his daughter, Liam must get a client to sign with the firm under false pretenses, Addy finally gets some attention from her boss when she speaks her mind, and Malcolm gets off on the wrong foot when he's hired outside of the firm's traditional process."""
1119634,ns/m.0kd3pbb,ns/tv.tv_series_episode.air_date,"""2010-01-21"""
1119635,ns/m.0kd3pbb,ns/tv.tv_series_episode.season_number,"""1"""


**Observation**: As one can see from the above example for the name "Pilot", it is full of Pilot episodes from various TV Shows.

## Unique Predicate Names

In [49]:
unique_predicates = freebase_df['predicate'].drop_duplicates().tolist()

In [50]:
len(unique_predicates)

533

## Unique Descriptions

In [51]:
description_df = freebase_df[freebase_df["predicate"] == "ns/common.topic.description"].drop_duplicates()

# List of unique descriptions, defined as the value (object) of the
# "ns/common.topic.description" (predicate) of the corresponding entity's ID (subject)
unique_descriptions = description_df["object"].drop_duplicates().tolist()

In [52]:
len(description_df)  # unique entity ID - description pairs

75844

In [53]:
len(unique_descriptions)

75655

In [54]:
len(description_df) - len(unique_descriptions) # non-unique descriptions

189

### Unique Entity ID to Description and vice-versa Dictionaries

In [55]:
id_to_description = description_df.set_index("subject")["object"].to_dict()

In [56]:
print("Number of unique IDs:")
print(len(id_to_name))

Number of unique IDs:
92128


In [57]:
description_to_id = {}
for _id, description in id_to_description.items():
    description_to_id.setdefault(description, []).append(_id)

In [58]:
print("Number of unique descriptions:")
print(len(description_to_id))

Number of unique descriptions:
75655


In [59]:
description_to_id_1to1 = {k: v[0] for k, v in description_to_id.items() if len(v) == 1}

In [60]:
print("Number of unique descriptions that correspond to single entity ID:")
print(len(description_to_id_1to1))

Number of unique descriptions that correspond to single entity ID:
75472


In [61]:
description_to_id_1toN = {k: v for k, v in description_to_id.items() if len(v) > 1}

In [62]:
print("Number of unique descriptions that correspond to multiple entity IDs:")
print(len(description_to_id_1toN))

Number of unique descriptions that correspond to multiple entity IDs:
183


In [63]:
description_to_id_1toN

{'"Patrick\'s parents are coming over and Patrick doesn\'t want to look stupid, so he tells SpongeBob to be stupider than him."': ['ns/m.07749b',
  'ns/m.0jb6zfd'],
 '"TBA"': ['ns/m.05dtyb4', 'ns/m.05dty9w'],
 '"Told by Robert Powell"': ['ns/m.0k1pdw2',
  'ns/m.0k1pdv6',
  'ns/m.0kng1fl',
  'ns/m.0k1pdtd',
  'ns/m.0k1pdvs'],
 '"New York 2012 and New York 1912, a hundred year gap, with something scarily in common, something unnatural, and whatever you do - do not blink."': ['ns/m.0lbsc1p',
  'ns/m.0lbsc48'],
 '"Finn and Jake go on a quest for a magical book that would prove them worthy of being righteous heroes."': ['ns/m.0gfk03c',
  'ns/m.0gfkc0h'],
 '"Finn and Princess Bubblegum must protect the Candy Kingdom from a horde of candy zombies they accidentally created."': ['ns/m.0gfk6h0',
  'ns/m.0gfx0m2'],
 '"Jake\'s plan to spend time with both Lady Rainicorn and Finn backfires."': ['ns/m.0gfvp0h',
  'ns/m.0kcpdm4'],
 '"Big money problems send Finn and Jake to Wildberry Kingdom."': ['ns

**"Told by Robert Powell" Example**

In [64]:
example_description_1toN = "Told by Robert Powell"
example_ids_1toN = description_to_id_1toN[f'"{example_description_1toN}"']
print(example_ids_1toN)

['ns/m.0k1pdw2', 'ns/m.0k1pdv6', 'ns/m.0kng1fl', 'ns/m.0k1pdtd', 'ns/m.0k1pdvs']


In [65]:
# Data Exploration
for example_id in example_ids_1toN:
    display(freebase_df.query(f'subject=="{example_id}"'))

,subject,predicate,object
26463,ns/m.0k1pdw2,ns/common.topic.description,"""Told by Robert Powell"""
26464,ns/m.0k1pdw2,ns/tv.tv_series_episode.air_date,"""1986-12-30"""
26465,ns/m.0k1pdw2,ns/common.topic.alias,"""The Turn of the Screw"""
26466,ns/m.0k1pdw2,ns/tv.tv_series_episode.season_number,"""0"""
26467,ns/m.0k1pdw2,ns/type.object.name,"""The Rose Garden"""
26468,ns/m.0k1pdw2,ns/tv.tv_series_episode.series,ns/m.05f4vlb
26469,ns/m.0k1pdw2,ns/tv.tv_series_episode.episode_number,"""8"""


,subject,predicate,object
26565,ns/m.0k1pdv6,ns/tv.tv_series_episode.air_date,"""1986-12-28"""
26566,ns/m.0k1pdv6,ns/type.object.name,"""Wailing Well"""
26567,ns/m.0k1pdv6,ns/common.topic.description,"""Told by Robert Powell"""
26568,ns/m.0k1pdv6,ns/tv.tv_series_episode.episode_number,"""6"""
26569,ns/m.0k1pdv6,ns/common.topic.alias,"""Number 13"""
26570,ns/m.0k1pdv6,ns/tv.tv_series_episode.season_number,"""0"""
26571,ns/m.0k1pdv6,ns/tv.tv_series_episode.series,ns/m.05f4vlb


,subject,predicate,object
26581,ns/m.0kng1fl,ns/tv.tv_series_episode.season_number,"""0"""
26582,ns/m.0kng1fl,ns/tv.tv_series_episode.series,ns/m.05f4vlb
26583,ns/m.0kng1fl,ns/type.object.name,"""The Ash Tree"""
26584,ns/m.0kng1fl,ns/common.topic.description,"""Told by Robert Powell"""
26585,ns/m.0kng1fl,ns/tv.tv_series_episode.episode_number,"""5"""
26586,ns/m.0kng1fl,ns/tv.tv_series_episode.air_date,"""1986-12-26"""


,subject,predicate,object
26617,ns/m.0k1pdtd,ns/common.topic.alias,"""The Stalls of Barchester"""
26618,ns/m.0k1pdtd,ns/tv.tv_series_episode.air_date,"""1986-12-25"""
26619,ns/m.0k1pdtd,ns/tv.tv_series_episode.episode_number,"""4"""
26620,ns/m.0k1pdtd,ns/type.object.name,"""The Mezzotinit"""
26621,ns/m.0k1pdtd,ns/tv.tv_series_episode.season_number,"""0"""
26622,ns/m.0k1pdtd,ns/tv.tv_series_episode.series,ns/m.05f4vlb
26623,ns/m.0k1pdtd,ns/common.topic.description,"""Told by Robert Powell"""


,subject,predicate,object
26624,ns/m.0k1pdvs,ns/tv.tv_series_episode.episode_number,"""7"""
26625,ns/m.0k1pdvs,ns/common.topic.alias,"""A Warning to the Curious"""
26626,ns/m.0k1pdvs,ns/common.topic.description,"""Told by Robert Powell"""
26627,ns/m.0k1pdvs,ns/tv.tv_series_episode.season_number,"""0"""
26628,ns/m.0k1pdvs,ns/tv.tv_series_episode.air_date,"""1986-12-29"""
26629,ns/m.0k1pdvs,ns/type.object.name,"""Oh, Whistle and I'll Come To You, My Lad"""
26630,ns/m.0k1pdvs,ns/tv.tv_series_episode.series,ns/m.05f4vlb


**Observation**: The description here is correct, but not distinguishing. These entities are episodes of Classic Ghost Stories told by Robert Powell. However, only the "Told by Robert Powell" is written in the description, giving no meaningful information about the entity.

**Episodes from "The Office" TV Show Example**

In [66]:
example_description_1toN = "Michael decides to open a cafe-disco in his old office. Pam and Jim are planning a secret trip."
example_ids_1toN = description_to_id_1toN[f'"{example_description_1toN}"']
print(example_ids_1toN)

['ns/m.05s_rz5', 'ns/m.05zt0y_']


In [67]:
# Data Exploration
for example_id in example_ids_1toN:
    display(freebase_df.query(f'subject=="{example_id}"'))

,subject,predicate,object
209890,ns/m.05s_rz5,ns/tv.tv_series_episode.director,ns/m.04t2l2
209891,ns/m.05s_rz5,ns/tv.tv_series_episode.writer,ns/m.0g9yd6_
209892,ns/m.05s_rz5,ns/tv.tv_series_episode.episode_number,"""23"""
209893,ns/m.05s_rz5,ns/common.topic.description,"""Michael decides to open a cafe-disco in his old office. Pam and Jim are planning a secret trip."""
209894,ns/m.05s_rz5,ns/tv.tv_series_episode.season_number,"""5"""
209895,ns/m.05s_rz5,ns/tv.tv_series_episode.air_date,"""2009-04-23"""
209896,ns/m.05s_rz5,ns/tv.tv_series_episode.next_episode,ns/m.05sxf44
209897,ns/m.05s_rz5,ns/tv.tv_series_episode.series,ns/m.08jgk1
209898,ns/m.05s_rz5,ns/type.object.name,"""Broke"""
209899,ns/m.05s_rz5,ns/freebase.valuenotation.is_reviewed,ns/m.01xrztx


,subject,predicate,object
221146,ns/m.05zt0y_,ns/type.object.name,"""Company Picnic"""
221147,ns/m.05zt0y_,ns/tv.tv_series_episode.season_number,"""5"""
221148,ns/m.05zt0y_,ns/tv.tv_series_episode.previous_episode,ns/m.05zlyff
221149,ns/m.05zt0y_,ns/tv.tv_series_episode.series,ns/m.08jgk1
221150,ns/m.05zt0y_,ns/tv.tv_series_episode.episode_number,"""26"""
221151,ns/m.05zt0y_,ns/common.topic.notable_types,ns/m.01xrzlb
221152,ns/m.05zt0y_,ns/tv.tv_series_episode.air_date,"""2009-05-14"""
221153,ns/m.05zt0y_,ns/common.topic.description,"""Michael decides to open a cafe-disco in his old office. Pam and Jim are planning a secret trip."""
221154,ns/m.05zt0y_,ns/tv.tv_series_episode.season,ns/m.0462vdy
221155,ns/m.05zt0y_,key/wikipedia.en,"""Company_Picnic"""


**Observation**: The description here is wrong and possibly there might be more such cases in Freebase. The description is accurate in the fact that it is about an episode from "The Office", however it is not either of the episode names shown above. This description suits better to episode [Cafe Disco](https://en.wikipedia.org/wiki/Cafe_Disco). Additionally, the episode numbers are incorrect. This means that Freebase or the max1024 subset as provided by the WikiGraphs paper may have inconsistent information.

# Vector Database

In [68]:
class VectorDatabase:
    def __init__(self, texts, vecdb_path, similarity="cosine"):
        """
        Initializes a VectorDatabase object for managing a vector database
        containing a list of given texts.

        This class stores and manages the given texts as word
        embeddings within a ChromaDB collection using a persistent storage.
        It provides functionality for querying and retrieving relevant
        texts based on similarity to a given query.

        Args:
            texts (list): A list of entity texts to be stored in the vector database.
            vecdb_path (str): The path to the directory where the ChromaDB collection
                will be stored.
            similarity (str, optional): The similarity metric to use for nearest neighbor search.
                Defaults to "cosine". Options include "cosine", "l2", "ip", or "dot".
        """
        self.texts = texts
        self.vecdb_path = vecdb_path
        self.similarity = similarity

        self.client = chromadb.PersistentClient(path=self.vecdb_path)
        self.collection_name = Path(self.vecdb_path).stem
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"hnsw:space": self.similarity}
        )
        self.ids = [f"id{i}" for i in range(1, len(texts) + 1)]

    def load_db(self, batch_size=None, reset=False):
        if reset:
            self.client.delete_collection(name=self.collection_name)
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"hnsw:space": self.similarity}
            )

        if self.collection.count() == 0:  # if collection is empty recreate it
            if type(batch_size) == int and batch_size > 0:
                if batch_size > 41666:
                    raise ValueError("batch_size must be less than 41666")
                for i in tqdm(range(0, len(self.texts), batch_size), desc="Creating VectorDB"):
                    batch_texts = self.texts[i : i + batch_size]
                    batch_ids = self.ids[i : i + batch_size]
                    self.collection.add(documents=batch_texts, ids=batch_ids)
            else:
                if len(self.texts) > 41666:
                    raise ValueError("Items to be added to collection must be "
                        "less than 41666. Please consider running in batches.")
                self.collection.add(documents=self.texts, ids=self.ids)

            print("VectorDB created at: " + self.vecdb_path)
        else:
            print("Loaded VectorDB from peristent storage")

    def query(self, query_text, n_results):
        results = self.collection.query(
            query_texts=[query_text],
            n_results=n_results
        )
        return results['documents']

## Names Vector Database

In [69]:
# Download and extract already created embeddings
download_file("https://drive.usercontent.google.com/download?id=1GmZrrVyUYamfv66NxTjVp7cvFlcb2U-v&export=download&confirm=t", "name_embeddings.tar.xz")
extract_tar("name_embeddings.tar.xz", save_path=".")

File name_embeddings.tar.xz already exists. Skipping download.
Skipping name_embeddings, already exists
Skipping name_embeddings/2d5ad6ea-6570-43c9-b284-1e733fd6d2e7, already exists
Skipping name_embeddings/2d5ad6ea-6570-43c9-b284-1e733fd6d2e7/data_level0.bin, already exists
Skipping name_embeddings/2d5ad6ea-6570-43c9-b284-1e733fd6d2e7/header.bin, already exists
Skipping name_embeddings/2d5ad6ea-6570-43c9-b284-1e733fd6d2e7/index_metadata.pickle, already exists
Skipping name_embeddings/2d5ad6ea-6570-43c9-b284-1e733fd6d2e7/length.bin, already exists
Skipping name_embeddings/2d5ad6ea-6570-43c9-b284-1e733fd6d2e7/link_lists.bin, already exists
Skipping name_embeddings/chroma.sqlite3, already exists
File name_embeddings.tar.xz extracted successfully.


In [70]:
name_embeddings = VectorDatabase(
    texts=unique_enitity_names,
    vecdb_path="name_embeddings"
)

In [71]:
name_embeddings.load_db(batch_size=1000)

Loaded VectorDB from peristent storage


### Example

In [72]:
name_embeddings.query("George", 10)  # top 10 example

[['"George"',
  '"George"',
  '"John George"',
  '"George Michael"',
  '"George VI"',
  '"George V"',
  '"George Martin"',
  '"George Best"',
  '"Harold L. George"',
  '"Georgecalvert"']]

In [73]:
name_embeddings.query("dog", 1)  # top 1 example

[['"Dog"']]

### Example based on a Parsed Pair

In [74]:
node_id = example_pair.center_node

In [75]:
id_to_name[node_id]

'"Valkyria Chronicles 3: Unrecorded Chronicles"'

In [76]:
# Using title to get the ID with the help of the vector database
title = parsed_pairs[0].title
name = name_embeddings.query(title, n_results=1)[0][0]
print(f"Query: {title}\nResult: {name}")

Query: Valkyria_Chronicles_III
Result: "Valkyria Chronicles 3: Unrecorded Chronicles"


In [77]:
name_to_id[name][0]

'ns/m.0ddd390'

## Descriptions Vector Database

In [78]:
# Download and extract already created embeddings
#download_file("https://drive.usercontent.google.com/download?id=1GmZrrVyUYamfv66NxTjVp7cvFlcb2U-v&export=download&confirm=t", "name_embeddings.tar.xz")
#extract_tar("name_embeddings.tar.xz", save_path=".")

In [79]:
description_embeddings = VectorDatabase(
    texts=unique_descriptions,
    vecdb_path="description_embeddings"
)

In [80]:
description_embeddings.load_db(batch_size=1000)

Loaded VectorDB from peristent storage


### Example

In [81]:
description_embeddings.query("George", 10)  # top 10 example

[['"George blows an invitation upstairs with his latest girlfriend and then when he tries to make restitution he leaves progressively nastier messages on her answering machine. He gets the chance to prevent her from hearing her messages by having Jerry switch the tape out of her machine, while he distracts her. Jerry and his girlfriend have a disagreement about a TV commercial for Dockers, and his telling his friends about their conflict."',
  '"George Sr. suggests a last-ditch effort to save the family by holding a fund raiser to raise money to pay their legal expenses. Michael suggests to Lindsay that she take over the house work, and he sends George-Michael to an expensive boarding school. Meanwhile, Maeby has trouble with a snowboarding film and Tobias tries to be a \\"discipline daddy\\"."',
  '"George IV was King of the United Kingdom of Great Britain and Ireland and of Hanover following the death of his father, George III, on 29 January 1820, until his own death ten years later.

In [82]:
description_embeddings.query("dog", 1)  # top 1 example

[['"The domestic dog is a domesticated canid which has been selectively bred for millennia for various behaviors, sensory capabilities, and physical attributes.\\nAlthough initially thought to have originated as a manmade variant of an extant canid species, extensive genetic studies undertaken during the 2010s indicate that dogs diverged from other wolf-like canids in Eurasia 40,000 years ago. Being the oldest domesticated animals, their long association with people has allowed dogs to be uniquely attuned to human behavior, as well as thrive on a starch-rich diet which would be inadequate for other canid species.\\nDogs perform many roles for people, such as hunting, herding, pulling loads, protection, assisting police and military, companionship, and, more recently, aiding handicapped individuals. This impact on human society has given them the nickname \\"man\'s best friend\\" in the Western world. In some cultures, however, dogs are also a source of meat."']]

## Predicate Vector Database

In [83]:
predicate_embeddings = VectorDatabase(
    texts=unique_predicates,
    vecdb_path="predicate_embeddings"
)

In [84]:
predicate_embeddings.load_db(batch_size=1000)

Loaded VectorDB from peristent storage


### Example

In [85]:
predicate_embeddings.query("parent of Google", 10)  # top 10 example

[['ns/organization.organization.parent',
  'ns/business.business_location.parent_company',
  'ns/organization.organization_relationship.parent',
  'ns/people.person.parents',
  'ns/organization.organization.child',
  'ns/people.family_member.family',
  'ns/organization.organization_relationship.child',
  'ns/base.wikipedia_infobox.video_game.developer',
  'ns/organization.organization_founder.organizations_founded',
  'ns/organization.organization.founders']]

In [86]:
predicate_embeddings.query("parent of John", 10)  # top 10 example

[['ns/people.person.parents',
  'ns/organization.organization.parent',
  'ns/people.family_member.family',
  'ns/organization.organization_relationship.parent',
  'ns/people.family.members',
  'ns/organization.organization_relationship.child',
  'ns/people.person.children',
  'ns/organization.organization.child',
  'ns/film.actor.film',
  'ns/people.person.place_of_birth']]

__Observation__: As one can notice from the above two examples, using the vector database, a descriptive text of a relation can be translated to the appropriate predicate. Here the Predicate Vector Database could understand the meaning of _Google_ and _John_ by finding as first result the `...organization.parent` and `...person.parents` predicates respectively, which are the correct ones.

## Backup Vector Databases

In [87]:
# create_tar("name_embeddings", ".", "xz")
# create_tar("predicate_embeddings", ".", "xz")
# create_tar("description_embeddings", ".", "xz")

## Restore Vector Databases

In [88]:
# extract_tar("name_embeddings.tar.xz", save_path=".")
# extract_tar("predicate_embeddings.tar.xz", save_path=".")
# extract_tar("description_embeddings.tar.xz", save_path=".")

# Generated Graph

The Generted Graph is created by prompting the LLM.

In [89]:
def graph_prompt(input_text: str, metadata={}, model="llama3:latest", silent=True, json_parse=True):
    SYS_PROMPT = (
        "You are a knowledge graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
        "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
            "\tTerms may include object, entity, location, organization, person, \n"
            "\tcondition, acronym, documents, service, concept, etc.\n"
            "\tTerms should be as atomistic as possible\n\n"
        "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
            "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
            "\tTerms can be related to many other terms\n\n"
        "Thought 3: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms"
        "and the relation between them, like the follwing: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]\n"
        #"When naming the nodes, use the same name for nodes that correspond to the same concept. Try to use the most descriptive and concise name.\n"
        "Do not add any other comment before or after the json. Respond ONLY with a well formed json that can be directly read by a program."
    )
    USER_PROMPT = f"context: ```{input_text}``` \n\n output: "
    response_dict = ollama.generate(model=model, system=SYS_PROMPT, prompt=USER_PROMPT)
    response = response_dict["response"]
    if not json_parse:
        return response
    try:
        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
    except json.decoder.JSONDecodeError as e:
        if not silent:
            print("\n\n JSON Parse ERROR ### Input : ", input_text[:50], "...\n\n")
        result = None
    except Exception as e:
        print("Unexpected Exception:\n", e)
    return result

## Split Text to Chunks
Splitting the document to smaller chunks, because when giving a big chunk to the LLM, it may produce
a summary of the given text rather than the requested graph as a JSON formatted text.

In [90]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

In [91]:
texts_pages = []  # list of page lists extracted from each text
for pair in parsed_pairs[:GENERATED_GRAPHS_SAMPLE_SIZE]:
    pages = splitter.split_text(pair.text)
    texts_pages.append(pages)

In [92]:
for i, pages in enumerate(texts_pages):
    print(f"###### ###### ######")
    print("Text title:", parsed_pairs[i].title)
    print("Number of Pages:", len(pages))
    print(f"###### ###### ######\n")
    for j, page in enumerate(pages, start=1):
        print(f"****** Page {j} ******:\n{page}\n****** ------ ******\n\n")

###### ###### ######
Text title: Valkyria_Chronicles_III
Number of Pages: 21
###### ###### ######

****** Page 1 ******:
= Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standar

### Drop Garbage Pages
Drop pages that are 50 characters in size or less to prevent creating false triplets, since these mostly include non meaningful text that could confuse the LLM

In [93]:
for i, pages in enumerate(texts_pages):
    print("Text title:", parsed_pairs[i].title)
    print("Number of Pages:", len(pages))
    dropped_pages = [page for page in pages if len(page) <= 50]
    texts_pages[i] = [page for page in pages if len(page) > 50]
    print("Garbage pages dropped:", len(dropped_pages), "\n")

Text title: Valkyria_Chronicles_III
Number of Pages: 21
Garbage pages dropped: 0 

Text title: Tower_Building_of_the_Little_Rock_Arsenal
Number of Pages: 21
Garbage pages dropped: 0 

Text title: Cicely_Mary_Barker
Number of Pages: 14
Garbage pages dropped: 0 

Text title: Plain_maskray
Number of Pages: 6
Garbage pages dropped: 0 

Text title: 2011$201312_Columbus_Blue_Jackets_season
Number of Pages: 19
Garbage pages dropped: 0 

Text title: Gregorian_Tower
Number of Pages: 8
Garbage pages dropped: 0 

Text title: There$0027s_Got_to_Be_a_Way
Number of Pages: 5
Garbage pages dropped: 0 

Text title: Nebraska_Highway_88
Number of Pages: 4
Garbage pages dropped: 0 

Text title: USS_Atlanta_$00281861$0029
Number of Pages: 16
Garbage pages dropped: 1 

Text title: Jacqueline_Fernandez
Number of Pages: 14
Garbage pages dropped: 0 



## Generating Subgraphs from Pages
After the generation procedure, the texts' subgraphs is saved as JSON in order to avoid repeating the LLM heavy processing on a second run of the notebook using the same LLM model.

It was chosen on purpose to save the texts' subgraphs before they are being parsed in case an error happens during parsing which would discard the whole progress. Also the benefit of saving the file after the subgraphs have been parsed is negligible, since the parsing process executes instantly.

In [94]:
texts_subgraphs_json_path = f"tmp/texts_subgraphs_{LLM_MODEL.replace(':', '-')}.json"
if not os.path.exists(texts_subgraphs_json_path):
    texts_subgraphs = []  # list of subgraph lists generated from each text
    for pages in tqdm(texts_pages):
        subgraphs = []
        for page in tqdm(pages, leave=False):
            subgraph = graph_prompt(page, model=LLM_MODEL, silent=False, json_parse=False)
            subgraphs.append(subgraph)
        texts_subgraphs.append(subgraphs)
    with open(texts_subgraphs_json_path, "w") as f:
        json.dump(texts_subgraphs, texts_subgraphs_json_path)
else:
    with open(texts_subgraphs_json_path) as f:
        texts_subgraphs = json.load(f)

## Cleaning Text

In [95]:
def fix_prompt_output(text):
    """Fixes the prompt output by removing text that would produce a malfromed JSON."""
    # any line not starting with these characters is an LLM comment and not a JSON text line
    starting_characters = ('"', '{', '}', '[', ']')
    lines = text.splitlines()
    filtered_lines = [line for line in lines if any(line.lstrip().startswith(char) for char in starting_characters)]
    return "\n".join(filtered_lines)

__NOTE__:

1. Using the first prompt (based on [knowledge graph][1]) without the added line for producing only well formed json, for this text input sample, it outputs 5 well formed JSON out of the 20 text inputs (i.e missing 15).
    1. Using the fix_prompt_output() function to filter the garbase text from the output text, by helping it conform to the JSON format, it outputs 20 out of 20 (i.e none missing).
3. Using the second prompt (based on both [knowledge graph][1] and [knowledge graph maker][2]), it still outputs 20 out of 20, but it also outputs more triplets. In particular 181 compared to 160 using the first prompt with fix_prompt_output(). Using the fix_prompt_output() function to the second prompt does no nothing, since the prompt produces correct JSON output without any filtering (at least for these 20 text inputs). In order to built a more robust solution however, the text inputs will be fed to fix_prompt_output() before being JSON parsed just in case a malformed output is produced.

[1]: https://github.com/rahulnyk/knowledge_graph/blob/main/helpers/prompts.py
[2]: https://github.com/rahulnyk/knowledge_graph_maker/blob/main/knowledge_graph_maker/graph_maker.py

In [96]:
fixed_texts_subgraphs = []
for subgraphs in texts_subgraphs:    
    fixed_subgraphs = []
    for subgraph in subgraphs:
        fixed_subgraph = fix_prompt_output(subgraph)
        fixed_subgraphs.append(fixed_subgraph)
    fixed_texts_subgraphs.append(fixed_subgraphs)

### Parsing Subgraphs and Merging Them

In [97]:
parsed_texts_subgraphs = []
for fixed_subgraphs in fixed_texts_subgraphs:  
    parsed_subgraphs = []
    for fixed_subgraph in fixed_subgraphs:
        try:
            parsed_subgraph = json.loads(fixed_subgraph)
        except json.decoder.JSONDecodeError as e:
            print(e)
            continue
        except Exception as e:
            print("Unexpected Exception:\n", e)
            continue
        parsed_subgraphs.append(parsed_subgraph)
    parsed_texts_subgraphs.append(parsed_subgraphs)

In [98]:
parsed_graphs = []
for parsed_subgraphs in parsed_texts_subgraphs:
    parsed_graph = list(chain(*parsed_subgraphs))  # join the subgraphs to create a whole graph
    parsed_graphs.append(parsed_graph)

In [99]:
parsed_graphs[-1]

[{'node_1': 'Jacqueline Fernandez',
  'node_2': 'Sri Lankan actress',
  'edge': 'is a'},
 {'node_1': 'Jacqueline Fernandez',
  'node_2': 'former model',
  'edge': 'was also a'},
 {'node_1': 'Miss Universe Sri Lanka',
  'node_2': 'Jacqueline Fernandez',
  'edge': 'represented her country at the 2006 world Miss Universe pageant'},
 {'node_1': 'Jacqueline Fernandez',
  'node_2': 'University of Sydney',
  'edge': 'graduated with a degree in mass communication from'},
 {'node_1': "Sujoy Ghosh's Aladin",
  'node_2': 'Jacqueline Fernandez',
  'edge': 'marked her acting debut as an actress'},
 {'node_1': "Mohit Suri's Murder 2",
  'node_2': 'Jacqueline Fernandez',
  'edge': "was Fernandez' breakthrough role, a commercial success and garnered her an IIFA Award for Best Supporting Actress nomination"},
 {'node_1': 'Housefull 2',
  'node_2': 'Jacqueline Fernandez',
  'edge': 'Fernandez played a glamorous role in the ensemble @-@ comedy'},
 {'node_1': 'IIFA Award for Star Debut of the Year – Femal

#### Wrongly Added Additional Properties by LLM

After doing a manual investigation, sometimes, the LLM may not be consistent and wrongly add additional properties (e.x like "relationType") that are outside of the standard triplets __node_1__, __node_2__ and __edge__.

These additional properties are found (if any) below and are removed.

In [100]:
for i, graph in enumerate(parsed_graphs):
    for triplet in graph:
        if len(triplet) > 3:
            illegal_properties = {k: v for k, v in triplet.items() if k not in ("node_1", "node_2", "edge")}
            for key in illegal_properties.keys():
                del triplet[key]  # Delete illegal property
            print(f"Graph {i}: Found these illegal properties: {illegal_properties}")
print("All illegal properties deleted (if any)")

Graph 2: Found these illegal properties: {'relationType': 'published in 1917'}
Graph 2: Found these illegal properties: {'relationType': 'published in 1920'}
All illegal properties deleted (if any)


In [101]:
parsed_graphs_dfs_raw = [pd.DataFrame(pg) for pg in parsed_graphs]

In [102]:
parsed_graphs_dfs_raw[0]

,node_1,node_2,edge
0,Valkyria Chronicles III,Senjō no Valkyria 3,alternate name for the game
1,Valkyria Chronicles III,PlayStation Portable,platform on which the game was developed and released
2,Valkyria Chronicles III,Valkyria series,part of a larger series of games
3,Valkyria Chronicles III,January 2011,release date in Japan
4,Nameless,penal military unit,type of military unit
...,...,...,...
202,two volumes,Senjō no Valkyria 3 : Namo naki Chikai no Hana,released in
203,2011 and 2012,Senjō no Valkyria 3 : Namo naki Chikai no Hana,between
204,Senjō no Valkyria 3 : Akaki Unmei no Ikusa Otome,Valkyria of the Battlefield 3,alternate adaptation
205,Mizuki Tsuge,Senjō no Valkyria 3 : Akaki Unmei no Ikusa Otome,illustrated by


#### Remove Duplicates

In [103]:
parsed_graphs_dfs_nodups = [df.drop_duplicates() for df in parsed_graphs_dfs_raw]

In [104]:
parsed_graphs_dfs_nodups[0]

,node_1,node_2,edge
0,Valkyria Chronicles III,Senjō no Valkyria 3,alternate name for the game
1,Valkyria Chronicles III,PlayStation Portable,platform on which the game was developed and released
2,Valkyria Chronicles III,Valkyria series,part of a larger series of games
3,Valkyria Chronicles III,January 2011,release date in Japan
4,Nameless,penal military unit,type of military unit
...,...,...,...
202,two volumes,Senjō no Valkyria 3 : Namo naki Chikai no Hana,released in
203,2011 and 2012,Senjō no Valkyria 3 : Namo naki Chikai no Hana,between
204,Senjō no Valkyria 3 : Akaki Unmei no Ikusa Otome,Valkyria of the Battlefield 3,alternate adaptation
205,Mizuki Tsuge,Senjō no Valkyria 3 : Akaki Unmei no Ikusa Otome,illustrated by


In [105]:
print("Number of Triplets of each parsed graph:")
print("\t\tRaw\tNo Duplicates\tDuplicates Removed")
for i, (df_raw, df_nodups) in enumerate(zip(parsed_graphs_dfs_raw, parsed_graphs_dfs_nodups)):
    print(f"Graph {i}: \t{len(df_raw)}\t{len(df_nodups)}\t\t{len(df_raw) - len(df_nodups)}")

Number of Triplets of each parsed graph:
		Raw	No Duplicates	Duplicates Removed
Graph 0: 	207	207		0
Graph 1: 	182	182		0
Graph 2: 	176	176		0
Graph 3: 	63	63		0
Graph 4: 	148	148		0
Graph 5: 	73	73		0
Graph 6: 	43	43		0
Graph 7: 	37	37		0
Graph 8: 	133	133		0
Graph 9: 	151	151		0


#### Generated Graph Creation

In [106]:
generated_graphs_dfs = parsed_graphs_dfs_nodups
generated_graphs_dfs_dict = {pair.center_node: df for pair, df in zip(parsed_pairs, generated_graphs_dfs)}

In [107]:
generated_df = pd.concat(generated_graphs_dfs_dict, names=["center_node"])

In [108]:
generated_graphs_dfs_dict['ns/m.0ddd390']

,node_1,node_2,edge
0,Valkyria Chronicles III,Senjō no Valkyria 3,alternate name for the game
1,Valkyria Chronicles III,PlayStation Portable,platform on which the game was developed and released
2,Valkyria Chronicles III,Valkyria series,part of a larger series of games
3,Valkyria Chronicles III,January 2011,release date in Japan
4,Nameless,penal military unit,type of military unit
...,...,...,...
202,two volumes,Senjō no Valkyria 3 : Namo naki Chikai no Hana,released in
203,2011 and 2012,Senjō no Valkyria 3 : Namo naki Chikai no Hana,between
204,Senjō no Valkyria 3 : Akaki Unmei no Ikusa Otome,Valkyria of the Battlefield 3,alternate adaptation
205,Mizuki Tsuge,Senjō no Valkyria 3 : Akaki Unmei no Ikusa Otome,illustrated by


In [109]:
print(parsed_pairs[7].text)


 = Nebraska Highway 88 = 
 
 Nebraska Highway 88 ( N @-@ 88 ) is a highway in northwestern Nebraska . It has a western terminus at Wyoming Highway 151 ( WYO 151 ) at the Wyoming – Nebraska state line . The road travels eastward to N @-@ 71 , where it turns south . N @-@ 88 continues east to south of Bridgeport . The road turns north , ends at an intersection with U.S. Highway 385 ( US 385 ) and N @-@ 92 in Bridgeport . The route was designated in 1937 , before the official state highway system was created . It was extended to the state line in 1986 . 
 
 = = Route description = = 
 
 N @-@ 88 starts at the Nebraska – Wyoming state line in Banner County , where WYO 151 ends , and travels northeast . The road quickly bends east after less than one mile ( 1 @.@ 6 km ) , and continues in a straight line . For the next twenty miles ( 32 km ) , N @-@ 88 intersects minor streets , through rural farmland . The route turns south at N @-@ 71 , and becomes concurrent . Four miles ( 6 @.@ 4 km ) 

In [110]:
parsed_graphs_dfs_raw[7]

,node_1,node_2,edge
0,Nebraska Highway 88,Wyoming Highway 151,has western terminus at
1,N @-@ 88,N @-@ 71,turns south to
2,N @-@ 88,Bridgeport,continues east to south of
3,N @-@ 88,U.S. Highway 385,ends at an intersection with
4,Nebraska Highway 88,1937,was designated in
5,Nebraska Highway 88,1986,was extended to the state line in
6,Nebraska – Wyoming,state line,located at
7,WYO 151,ends,terminates at
8,N @-@ 88,travels northeast,direction
9,farmland,passes through,location


In [111]:
generated_graphs_dfs[7]

,node_1,node_2,edge
0,Nebraska Highway 88,Wyoming Highway 151,has western terminus at
1,N @-@ 88,N @-@ 71,turns south to
2,N @-@ 88,Bridgeport,continues east to south of
3,N @-@ 88,U.S. Highway 385,ends at an intersection with
4,Nebraska Highway 88,1937,was designated in
5,Nebraska Highway 88,1986,was extended to the state line in
6,Nebraska – Wyoming,state line,located at
7,WYO 151,ends,terminates at
8,N @-@ 88,travels northeast,direction
9,farmland,passes through,location


In [112]:
generated_df

node_1                node_2  \
center_node                                                       
ns/m.0ddd390 0    Valkyria Chronicles III   Senjō no Valkyria 3   
             1    Valkyria Chronicles III  PlayStation Portable   
             2    Valkyria Chronicles III       Valkyria series   
             3    Valkyria Chronicles III          January 2011   
             4                   Nameless   penal military unit   
...                                   ...                   ...   
ns/m.02q1mmh 146                     Elle             magazines   
             147                    Verve             magazines   
             148         Harper 's Bazaar             magazines   
             149           Women's Health             magazines   
             150               L'Officiel             magazines   

                                                                   edge  
center_node                                                              
ns/m.0ddd390 0                              alternate name for the game  
             1    platform on which the game was developed and released  
             2                         part of a larger series of games  
             3                                    release date in Japan  
             4                                    type of military unit  
...                                                                 ...  
ns/m.02q1mmh 146                cover model for many Indian editions of  
             147                cover model for many Indian editions of  
             148                cover model for many Indian editions of  
             149                cover model for many Indian editions of  
             150                cover model for many Indian editions of  

[1213 rows x 3 columns]

## Data Exploration

### Partial Names

#### Jacqueline Fernandez Example

In [113]:
partial_names_example_graph = parsed_graphs[-1]
partial_names_example_graph

[{'node_1': 'Jacqueline Fernandez',
  'node_2': 'Sri Lankan actress',
  'edge': 'is a'},
 {'node_1': 'Jacqueline Fernandez',
  'node_2': 'former model',
  'edge': 'was also a'},
 {'node_1': 'Miss Universe Sri Lanka',
  'node_2': 'Jacqueline Fernandez',
  'edge': 'represented her country at the 2006 world Miss Universe pageant'},
 {'node_1': 'Jacqueline Fernandez',
  'node_2': 'University of Sydney',
  'edge': 'graduated with a degree in mass communication from'},
 {'node_1': "Sujoy Ghosh's Aladin",
  'node_2': 'Jacqueline Fernandez',
  'edge': 'marked her acting debut as an actress'},
 {'node_1': "Mohit Suri's Murder 2",
  'node_2': 'Jacqueline Fernandez',
  'edge': "was Fernandez' breakthrough role, a commercial success and garnered her an IIFA Award for Best Supporting Actress nomination"},
 {'node_1': 'Housefull 2',
  'node_2': 'Jacqueline Fernandez',
  'edge': 'Fernandez played a glamorous role in the ensemble @-@ comedy'},
 {'node_1': 'IIFA Award for Star Debut of the Year – Femal

In [114]:
print(parsed_pairs[GENERATED_GRAPHS_SAMPLE_SIZE - 1].text)


 = Jacqueline Fernandez = 
 
 Jacqueline Fernandez ( born 11 August 1985 ) is a Sri Lankan actress , former model , and the winner of the 2006 Miss Universe Sri Lanka pageant . As Miss Universe Sri Lanka she represented her country at the 2006 world Miss Universe pageant . She graduated with a degree in mass communication from the University of Sydney , and worked as a television reporter in Sri Lanka . 
 While on a modelling assignment in India in 2009 , Fernandez successfully auditioned for Sujoy Ghosh 's fantasy drama Aladin , which marked her acting debut . Fernandez ' breakthrough role was in Mohit Suri 's psychological thriller Murder 2 ( 2011 ) , her first commercial success . This was followed by glamorous roles in the ensemble @-@ comedy Housefull 2 ( 2012 ) and its sequel Housefull 3 , and the action thriller Race 2 ( 2013 ) , all of which were box @-@ office successes . Her performance in the first of these garnered her an IIFA Award for Best Supporting Actress nomination .

## Standardization

In this step of the process, the descriptive names given by the LLM will be converted to the standard names as defined by Freebase, with the help of the Names and Predicates Vector Databases.